In [1]:
from pandas import DataFrame
import numpy as np
from math import log2
import random
from sklearn.tree import DecisionTreeRegressor as DT

In [2]:
def relNormalize1(rel, max_rel=19):
    norm_rel = rel - min(rel)
    if max(norm_rel) != 0:
        norm_rel = norm_rel * max_rel / max(norm_rel) + 1
    return norm_rel

def relNormalize2(rel):
    uniq_rel = np.unique(rel)
    uniq_rel = sorted(uniq_rel)
    norm_rel = np.empty(rel.shape)
    for i, val in enumerate(uniq_rel):
        norm_rel[rel==val] = i + 1
    return norm_rel * 5


class QueryData:
    def __init__(self, X, rel):
        self._X = np.copy(X)
        self._n = self._X.shape[0]
        self._rel = np.copy(rel)

        right_order = np.argsort(self._rel)[::-1]
        self._X = self._X[right_order]
        self._rel = self._rel[right_order]
        self._2_rel = 2 ** self._rel - 1
        self._log2_order = np.log2(np.array(range(self._n)) + 2)

    def maxDCG(self, T_NDCG):
        return sum(self._2_rel[:T_NDCG] / self._log2_order[:T_NDCG])

    def getX(self):
        return self._X

    def getY(self):
        return self._rel

    def getSwapNDCGMatrix(self, rel, T_NDCG):
        order = np.empty(rel.shape)
        sort_rel = sorted(zip(rel, range(rel.shape[0])), key=lambda x: x[0], reverse=True)
        order[list(map(lambda x: x[1], sort_rel))] = range(1, rel.shape[0] + 1)

        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        matr_elem_DCG = np.tile(elem_DCG, (self._n, 1)).T
        matr_swap_elem_DCG = (self._2_rel.reshape((1, self._n)) / log2_order.reshape((self._n, 1))).T

        lambda_mtr = - matr_elem_DCG - matr_elem_DCG.T + matr_swap_elem_DCG + matr_swap_elem_DCG.T
        no_null_swap = ((order <= T_NDCG).reshape((self._n, 1)) + (order <= T_NDCG).reshape((1, self._n))) > 0
        lambda_mtr = np.abs(lambda_mtr * no_null_swap)
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG != 0:
            return lambda_mtr / max_DCG
        else:
            return lambda_mtr

    def getNDCG(self, rel, T_NDCG):
        order = np.empty(rel.shape)
        sort_rel = sorted(zip(rel, range(rel.shape[0])), key=lambda x: x[0], reverse=True)
        order[list(map(lambda x: x[1], sort_rel))] = range(1, rel.shape[0] + 1)
        log2_order = np.log2(order + 1)
        elem_DCG = self._2_rel / log2_order
        DCG = sum(elem_DCG[order <= T_NDCG])
        max_DCG = self.maxDCG(T_NDCG)
        if max_DCG != 0:
            return DCG / max_DCG
        else:
            return DCG

    def getCountErrorPair(self, rel):
        _n = rel.shape[0]
        pairs = (rel.reshape((_n, 1)) - rel.reshape((1, _n)) <= 0)
        true_pairs = (self._rel.reshape((_n, 1)) - self._rel.reshape((1, _n)) > 0)
        count = sum(sum(pairs == true_pairs))
        return count


class LambdaRankTrees:
    def __init__(self, learning_rate=0.5, n_estimators=1000, sigma=1, start_depth=5):
        self._learning_rate = learning_rate
        self._n_estimators = n_estimators
        self._sigma = sigma
        self._start_depth = start_depth
        self._trees = None

    def _getTrainX(self, queries_data):
        n = 0
        m = queries_data[0].getX().shape[1]
        for query_data in queries_data:
            n += query_data._n

        X = np.empty((n, m), dtype=np.float64)
        Y = np.empty(n, dtype=np.float64)
        indexs = []
        cur_index = 0
        for query_data in queries_data:
            cur_n = query_data._n
            X[cur_index:cur_index + cur_n] = query_data.getX()
            Y[cur_index:cur_index + cur_n] = query_data.getY()
            indexs.append(range(cur_index, cur_index + cur_n))
            cur_index += cur_n
        return X, Y, indexs

    def _getGradient(self, queries_data, h, indexs_data, T_NDCG):
        g = np.empty(h.shape[0], dtype=np.float64)
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            rel_n = rel.shape[0]
            
            delta_rels = rel.reshape((rel_n, 1)) - rel.reshape((1, rel_n))
            sign_matr = np.sign(query_data._rel.reshape((rel_n, 1)) - query_data._rel.reshape((1, rel_n)))
            
            lambda_matr = - self._sigma / (1 + np.exp(self._sigma * (sign_matr * delta_rels))) #* query_data.getSwapNDCGMatrix(rel, T_NDCG)
            lambda_vector = np.sum(lambda_matr * sign_matr, axis=1)
            g[indexs] = lambda_vector
        return g

    def _getNDCG(self, queries_data, h, indexs_data, T_NDCG):
        ndcg = 0
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            ndcg += query_data.getNDCG(rel, T_NDCG)
        return ndcg / len(indexs_data)

    def _countErrorPair(self, queries_data, h, indexs_data):
        count = 0
        for i, indexs in enumerate(indexs_data):
            query_data = queries_data[i]
            rel = h[indexs]
            count += query_data.getCountErrorPair(rel)
        return count

    def fit(self, queries_data, persent_valid=0.2, persent_train=0.1):
        random.seed(1234)
        random.shuffle(queries_data)
        count_valid = int(persent_valid * len(queries_data))
        data_valid = queries_data[:count_valid]
        data_train = queries_data[count_valid:]
        #data_train = queries_data

        X_train, y_train, index_train = self._getTrainX(data_train)
        X_valid, y_valid, index_valid = self._getTrainX(data_valid)
        self._trees = []
        self._score_train = []
        self._score_valid = []

        h_train = np.zeros(X_train.shape[0])
        h_valid = np.zeros(X_valid.shape[0])
        for iteration in range(self._n_estimators):
            print(iteration, self._countErrorPair(data_train, h_train, index_train),
                 self._getNDCG(data_train, h_train, index_train, 5),
                    self._countErrorPair(data_valid, h_valid, index_valid),
                 self._getNDCG(data_valid, h_valid, index_valid, 5))
                  #self._countErrorPair(data_valid, h_valid, index_valid))

            g = self._getGradient(data_train, h_train, index_train, 5)
            norm_g = np.linalg.norm(g)
            d_tree = DT(max_depth=self._start_depth)
            d_tree.fit(X_train, -g)
            self._trees.append(d_tree)
            h_train += self._learning_rate * d_tree.predict(X_train)
            h_valid += self._learning_rate * d_tree.predict(X_valid)

            # self._score_train.append(self._getNDCG(data_train, h_train, index_train, 5))
            # self._score_valid.append(self._getNDCG(data_valid, h_valid, index_valid, 5))

    def predict(self, X):
        y = np.zeros(X.shape[0])
        for tree in self._trees:
            y += self._learning_rate * tree.predict(X)
        return y

In [3]:
def loadData():
    trainPath = "../data/train.data.cvs"
    return DataFrame.from_csv(trainPath, index_col=False).as_matrix()

def loadSmallData():
    return np.load("small_train.npy")

def saveResults(queries, rels, namefile):
    uniq_queries = np.unique(queries)
    ans = np.empty((rels.shape[0], 2), dtype=np.int)
    count_last = 1
    for q in uniq_queries:
        rel = rels[queries == q]
        order = np.argsort(rel)[::-1] + count_last
        ans[queries == q, 0] = order
        ans[queries == q, 1] = q
        count_last += rel.shape[0]
    df = DataFrame(ans, columns=["DocumentId","QueryId"])
    df.to_csv(open(namefile, "w"), index=False)

In [4]:
rowData = loadData()
queries = rowData[:, -1]
uniq_queries = np.unique(queries)
queries_train_data = []
for q in uniq_queries:
    xy = rowData[queries == q][:, :-1]
    queries_train_data.append(QueryData(xy[:, 1:], relNormalize2(xy[:, 0])))

In [5]:
lambdaRank = LambdaRankTrees(learning_rate=0.1, n_estimators=100, sigma=1, start_depth=3)
lambdaRank.fit(queries_train_data[:120],  persent_valid=0.2, persent_train=1)

0 399473 1.0 90192 1.0
1 487186 0.533776922199 118954 0.519584561941
2 515663 0.418377136398 128709 0.305555980655
3 534854 0.310619018379 139662 0.247833237414
4 554289 0.287308435629 145494 0.180033614185
5 558152 0.305391162919 146105 0.172047591539
6 563510 0.310972838815 147089 0.182001643566
7 562327 0.324291592039 146570 0.182740821549
8 572685 0.322599230749 149363 0.199512361289
9 572657 0.319773661693 150411 0.183796296259
10 569199 0.321288210565 150364 0.176241442038
11 569176 0.303841611092 151335 0.150391362026
12 566094 0.29228471327 151599 0.150660018101
13 564448 0.28850971441 152366 0.148324758515
14 560442 0.308838636885 151871 0.181408298993
15 558741 0.315289773458 152178 0.153269906116
16 556063 0.295094631082 152286 0.145730947229
17 554518 0.295777843657 152592 0.136793629678
18 552815 0.300542527124 152734 0.139305687676
19 551484 0.298487354992 152568 0.139298616342
20 550434 0.297111576212 153181 0.140132892235
21 549530 0.305230316704 153659 0.137317770241
2

In [18]:
testRow  = DataFrame.from_csv("../data/testset.cvs", index_col=False).as_matrix()

In [19]:
ans = lambdaRank.predict(testRow[:, 1:-1])
saveResults(testRow[:, -1], ans, "result9")